In [1]:
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt

## Comparison of coverage

The paper, "Confidence limits for prevalence of disease adjustedfor estimated sensitivity and specificity" by Lang and Reiczigel propose an approximate approach to produce confidence intervals.

In their table 1, they show that for true specificity 0.99 and true prevalence near 0.02, the method produces 95% confidence intervals that significantly overcover (coverage is about 99.4%). Since this is the domain pertenant to SARS-CoV-2 sero-prevalence, it is possible that this approach produces confidence intervals that are larger than necessary.

In the following, I confirm the coverage calculation using the calculator:
http://www2.univet.hu/users/jreiczig/CI4prevSeSp/calc02/index.php

 * True sensitivity = 0.99
 * True specificity = 0.99
 * n_se = 100
 * n_sp = 100

 * True prevalence = 0.02

 * n_p = 100

In [2]:
# Lang and Reiczigel method:
# online calculator produces 95% CL intervals that contain the true prevalence (0.02) for indicated range of positives
# (indexed by the number of false positives) - it does not depend on the observed number of false negatives
ranges = {
    0:[0,7],
    1:[0,8],
    2:[0,10],
    3:[0,12],
    4:[1,13],
    5:[1,15],
    6:[2,16]
}

n_p = 100
n_sp = 100

# probability of a positive = true prevalence*sensitivity + false positive rate
prev = 0.02
sensitivity = 0.99
p_fp = 0.01
p = prev*sensitivity + p_fp

# consider the possible outcomes - tally the probability that the true prevalence is in the reported 95% CL intervals.

prob_in_interval = 0.

for k_fp in range(7):
    # probability that the specificity test results in k_fp false positives
    prob_kfp = stats.binom.pmf(k_fp,n_sp,p_fp)
    
    # work out probability, given k_fp, that the number of positives will produce CI containing true prevalence
    upper = ranges[k_fp][1]
    prob_kp = stats.binom.cdf(upper,n_p,p)
    lower = ranges[k_fp][0]
    if lower > 0:
        prob_kp -= stats.binom.cdf(lower-1,n_p,p)
        
    # joint probability for k_fp and CI contains true prevalence:
    product = prob_kfp * prob_kp
    prob_in_interval += product
    
print(prob_in_interval)
print('(Table 1 shows this value as 0.994)')

0.9940491846455418
(Table 1 shows this value as 0.994)


## Compare with approach based on Bayesian inference of false positive rate

Prior density for false positive rate is assumed to be uniform.

Given a value for k_fp (the number of negative samples falsely identified as positive) the posterior probability density for the false positive probability (eta) is proporational to the likelihood(eta) = binom.pmf(k_fp, n_fp, eta).

Use that posterior to represent the probability distribution for the false positive rate.

In [3]:
# draw a fpr from the posterior distribution (acceptance rejection)
def get_fpr(i_fp, i_control):
    nu = 1.*i_fp/i_control
    big = stats.binom.pmf(i_fp, i_control, nu)
    max_fpr = 0.2 # fpr will be restricted to be below this value
    fpr_result = -1.
    while fpr_result < 0.:
        fpr_trial = stats.uniform.rvs(0.,max_fpr)
        pdf = stats.binom.pmf(i_fp, i_control, fpr_trial)
        if stats.uniform.rvs(0.,big) < pdf:
            fpr_result = fpr_trial
    return fpr_result

In [10]:
prev = 0.02
p = prev*sensitivity + p_fp

cl = 0.95
upper = 1.-(1-cl)/2.
lower = (1-cl)/2.

n_reps = 1000
i_upper = int(upper*n_reps)
i_lower = int(lower*n_reps)

prob_in_interval = 0.

print('fp low high')

for k_fp in range(7):
    # probability that the specificity test results in k_fp false positives
    prob_kfp = stats.binom.pmf(k_fp,n_sp,p_fp)

    # work out the distribution of observations for k_p
    results = []
    for i in range(n_reps):
        fpr = get_fpr(k_fp, n_sp)
        prob = prev*sensitivity + fpr
        results.append(stats.binom.rvs(n_p, prob))
        
    results.sort()
    upper = results[i_upper]
    lower = results[i_lower]
    print(k_fp,lower,upper)
    
    prob_kp = stats.binom.cdf(upper,n_p,p)
    if lower > 0:
        prob_kp -= stats.binom.cdf(lower-1,n_p,p)
        
    # joint probability for k_fp and CI contains true prevalence:
    product = prob_kfp * prob_kp
    prob_in_interval += product
    
print('coverage: ',prob_in_interval)

fp low high
0 0 8
1 0 10
2 1 11
3 1 14
4 2 14
5 2 16
6 3 18
coverage:  0.9830613479681555


## Findings

Both methods overcover. The overcoverage of the L&R method is more severe. The tables shows for different prevalences (prev) the coverage for the 95% CL intervals by the two methods considered.

prev | L&R | Bayes
---|---|---
0.005 | 0.994 | 0.994
0.01 | 0.994 | 0.996
0.02 | 0.994 | 0.983
0.03 | 0.992 | 0.974
0.05 | 0.984 | 0.972